In [1]:
import os
import numpy as np
import efficientnet.tfkeras
from tensorflow.keras.models import load_model
from tensorflow.keras import models
import tensorflow as tf
from focal_loss import BinaryFocalLoss

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"]=""

In [3]:
model_dir ='/media/tohn/SSD/ModelTrainByImages/R2_6/models/B5R2b5_AN_1FC_fold6_3.h5'
model = load_model(model_dir)
height = width = model.input_shape[1]

x = model.get_layer('head_pooling').output
prediction_layer = model.output
model2 = models.Model(inputs= model.input, outputs=[prediction_layer,x]) 

In [4]:
#ไม่ต้องเปลี่ยนก็ได้ เพราะเอา Label มาใช้เฉยๆ 
# validation
import pandas as pd
base_dir  = '/media/tohn/SSD/Images/Image2/validation/'
dataframe = pd.read_csv( '/home/yupaporn/codes/USAI/Validationdf_fold1_3.csv')
validation_dir = os.path.join(base_dir, 'validation')

#Train
train_df = pd.read_csv( '/home/yupaporn/codes/USAI/Traindf_fold1_3.csv')
base_dir0 = '/media/tohn/SSD/Images/Image1/'
os.chdir(base_dir0)
train_dir = os.path.join(base_dir0, 'train')

In [5]:
import pandas as pd
df0 = pd.read_csv (r'/home/yupaporn/codes/USAI/Validationdf_fold6_3.csv')
print(df0 .shape)
dataframe = df0[(df0['Path Crop']!='None' )&(df0['Path Crop']!='Nan')]
# a = df01[df01['Case'].between(1, 10)]
# dataframe = df01[df01['Abs Position']!='P8']
print(dataframe.shape)
print('Normal: ',dataframe[dataframe['Class']=='Normal'].shape)
print('Abnormal: ',dataframe[dataframe['Class']=='Abnormal'].shape)
dataframe.head(5)

(1035, 22)
(1035, 22)
Normal:  (585, 22)
Abnormal:  (450, 22)


,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Case,Abs Position,Sub Position,Class,Sub_class,Path Full,Path Crop,...,tagName,originalImage,left,top,width,height,Rleft,Rtop,Rwidth,Rheight
0,0,136,136,13,P1,P1,Abnormal,AB01,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,...,MildFattyLiver,https://irisprodseatraining.blob.core.windows....,274.0,159.0,432.0,415.0,0.287594,0.176762,0.453008,0.462151
1,1,140,140,13,P1,P1,Abnormal,AB02,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,...,ModerateFattyLiver,https://irisprodseatraining.blob.core.windows....,302.0,121.0,376.0,372.0,0.320609,0.135480,0.398844,0.414141
2,2,144,144,13,P1,P1,Abnormal,AB03,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,...,SevereFattyLiver,https://irisprodseatraining.blob.core.windows....,209.0,189.0,528.0,692.0,0.213301,0.210227,0.537893,0.769697
3,3,150,150,13,P1,P1,Abnormal,AB04,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,...,Cirrhosis,https://irisprodseatraining.blob.core.windows....,329.0,123.0,613.0,459.0,0.329811,0.137500,0.614545,0.511111
4,4,152,152,13,P1,P1,Abnormal,AB06,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,...,PDF2,https://irisprodseatraining.blob.core.windows....,291.0,101.0,624.0,500.0,0.292160,0.113258,0.626594,0.555556


In [6]:
# dataframe.info()

In [6]:
batch_size = 64
epochs = 10

from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=30,
      width_shift_range=0.2,
      height_shift_range=0.2,
      brightness_range=[0.5,1.5],
      shear_range=0.4,
      zoom_range=0.2,
      horizontal_flip=False,
      fill_mode='nearest')

train_generator = train_datagen.flow_from_dataframe(
        dataframe = dataframe,
        directory = train_dir,
        x_col = 'Path Crop',
        y_col = 'Class',
        target_size = (height, width),
        batch_size=batch_size,
        color_mode= 'rgb',
        class_mode='categorical')

#label
labels = (train_generator.class_indices)
labels = dict((v,k.replace("C","")) for k,v in labels.items())
print(labels)

Found 1035 validated image filenames belonging to 2 classes.
{0: 'Abnormal', 1: 'Normal'}


In [7]:
from tensorflow.keras.preprocessing import image
def predict_image(img_path):
    # Read the image and resize it
    img = image.load_img(img_path, target_size=(height, width))
    # Convert it to a Numpy array with target shape.
    x = image.img_to_array(img)
    # Reshape
    x = x.reshape((1,) + x.shape)
    x /= 255.
    result = model2.predict([x])
    result1 = result[0][0]
    result2 = result[1][0]

    return result1, result2

#Predict
pred_list = list()
prob_list = list()
vectorlist = list()
img_path=dataframe['Path Crop'].tolist()
for i in range(0,len(img_path)):
    result1,result2 = predict_image(img_path[i])
    result = np.argmax(result1)
    pred_list.append(labels[result])
    prob_list.append(result1[result])
    vectorlist.append(result2.tolist())
    
dataframe['category'] = pred_list
dataframe['Prob'] = prob_list

In [8]:
vector_array = np.array(vectorlist)
vector_array

array([[-0.05590712, -0.05188219, -0.04027781, ...,  0.19106752,
         0.08318286, -0.07087535],
       [-0.15836139, -0.17973068,  0.28286743, ..., -0.11856713,
         0.29551035, -0.10555316],
       [-0.19035314, -0.16670041,  0.17306362, ..., -0.03797427,
         0.30812865, -0.1265021 ],
       ...,
       [ 0.36495525,  0.50260156, -0.23210357, ...,  0.43507254,
        -0.22722208,  0.26264527],
       [-0.04337978,  0.17067377, -0.22293776, ...,  0.06723592,
        -0.20322663,  0.2319133 ],
       [-0.06137575,  0.41825467, -0.20861025, ...,  0.63882655,
        -0.20446403,  0.33317783]])

In [9]:
dataset = pd.DataFrame(vector_array[:, :])
dataset

,0,1,2,3,4,5,6,7,8,9,...,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047
0,-0.055907,-0.051882,-0.040278,0.264976,-0.087209,-0.196564,-0.056968,-0.128653,-0.039314,-0.217447,...,-0.096194,-0.185646,-0.095403,0.024173,-0.091600,-0.206813,-0.136027,0.191068,0.083183,-0.070875
1,-0.158361,-0.179731,0.282867,0.260556,-0.198368,0.054077,0.093965,-0.168819,-0.176397,0.003781,...,-0.182329,0.195726,-0.210673,-0.154106,-0.164888,-0.155402,0.145854,-0.118567,0.295510,-0.105553
2,-0.190353,-0.166700,0.173064,0.336238,-0.213590,0.169006,0.044973,-0.150350,-0.207814,0.172019,...,-0.168197,0.161285,-0.134653,-0.166482,-0.139779,0.162308,0.039460,-0.037974,0.308129,-0.126502
3,-0.030163,-0.160920,-0.025277,0.444823,-0.168903,0.170418,0.557955,-0.083914,-0.165666,-0.098059,...,-0.024460,0.109737,-0.134408,0.043680,-0.154191,0.250303,0.513003,-0.088720,0.424203,-0.158501
4,-0.141455,-0.174257,-0.127677,0.272662,-0.162064,0.148980,0.395592,-0.019139,-0.127176,-0.037056,...,-0.141242,0.112641,-0.032107,-0.165936,-0.165763,0.232291,0.260357,-0.130044,0.420623,-0.160887
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1030,-0.122954,0.209771,-0.216331,-0.064288,0.063501,-0.202775,-0.201548,0.080951,-0.009770,-0.158095,...,-0.025426,-0.122926,0.168830,0.119322,-0.054826,-0.171935,-0.072377,0.016204,-0.193050,0.011045
1031,-0.120600,0.116369,-0.199377,-0.086031,-0.066019,-0.181562,-0.165780,-0.149596,0.060642,-0.136027,...,0.032067,-0.144833,-0.191355,-0.131764,-0.085272,-0.098684,-0.098271,0.031458,-0.118339,-0.103795
1032,0.364955,0.502602,-0.232104,-0.163024,0.687879,-0.200081,-0.212717,0.134577,0.429816,-0.238719,...,0.120141,-0.231302,0.112709,0.443061,0.812059,-0.205081,-0.150918,0.435073,-0.227222,0.262645
1033,-0.043380,0.170674,-0.222938,-0.189437,0.218148,-0.151236,-0.187092,0.114719,0.169700,-0.102423,...,-0.055468,-0.172613,0.366476,-0.038488,-0.008692,-0.169950,-0.161646,0.067236,-0.203227,0.231913


In [10]:
a = dataframe['Class'].tolist()
b = dataframe['Sub Position'].tolist()
c = dataframe['Views'].tolist()
d = dataframe['Sub_class'].tolist()
dataset['Class'] = a 
dataset['SubPosition'] = b
dataset['Views'] = c
dataset['Sub_class'] = d

In [11]:
dataset

,0,1,2,3,4,5,6,7,8,9,...,2042,2043,2044,2045,2046,2047,Class,SubPosition,Views,Sub_class
0,-0.055907,-0.051882,-0.040278,0.264976,-0.087209,-0.196564,-0.056968,-0.128653,-0.039314,-0.217447,...,-0.091600,-0.206813,-0.136027,0.191068,0.083183,-0.070875,Abnormal,P1,FP-A,AB01
1,-0.158361,-0.179731,0.282867,0.260556,-0.198368,0.054077,0.093965,-0.168819,-0.176397,0.003781,...,-0.164888,-0.155402,0.145854,-0.118567,0.295510,-0.105553,Abnormal,P1,FP-A,AB02
2,-0.190353,-0.166700,0.173064,0.336238,-0.213590,0.169006,0.044973,-0.150350,-0.207814,0.172019,...,-0.139779,0.162308,0.039460,-0.037974,0.308129,-0.126502,Abnormal,P1,FP-A,AB03
3,-0.030163,-0.160920,-0.025277,0.444823,-0.168903,0.170418,0.557955,-0.083914,-0.165666,-0.098059,...,-0.154191,0.250303,0.513003,-0.088720,0.424203,-0.158501,Abnormal,P1,FP-A,AB04
4,-0.141455,-0.174257,-0.127677,0.272662,-0.162064,0.148980,0.395592,-0.019139,-0.127176,-0.037056,...,-0.165763,0.232291,0.260357,-0.130044,0.420623,-0.160887,Abnormal,P1,FP-A,AB06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1030,-0.122954,0.209771,-0.216331,-0.064288,0.063501,-0.202775,-0.201548,0.080951,-0.009770,-0.158095,...,-0.054826,-0.171935,-0.072377,0.016204,-0.193050,0.011045,Normal,P8,FP-E,Normal
1031,-0.120600,0.116369,-0.199377,-0.086031,-0.066019,-0.181562,-0.165780,-0.149596,0.060642,-0.136027,...,-0.085272,-0.098684,-0.098271,0.031458,-0.118339,-0.103795,Normal,P8,FP-E,Normal
1032,0.364955,0.502602,-0.232104,-0.163024,0.687879,-0.200081,-0.212717,0.134577,0.429816,-0.238719,...,0.812059,-0.205081,-0.150918,0.435073,-0.227222,0.262645,Normal,P8,FP-E,Normal
1033,-0.043380,0.170674,-0.222938,-0.189437,0.218148,-0.151236,-0.187092,0.114719,0.169700,-0.102423,...,-0.008692,-0.169950,-0.161646,0.067236,-0.203227,0.231913,Normal,P8,FP-E,Normal


In [12]:
os.chdir( '/home/yupaporn/codes/USAI/')
testdf = dataset
testdf.to_csv('FVvalidation_AN_RF_AN_fold6_3.csv')